# MHC-Fine Colab

Change log:
- March 7:
  - added upgrade for gdown to deal with model loading issue
  - updated mhc-fine repo to fix issue with np.object and np.int
  - added new MSA generation and fixed the naming issue
- March 18:
  - updated model to fix some minor issues

## Setup

In [2]:
import os
if not os.path.exists('mhc-fine'):
  os.system("git clone https://bitbucket.org/abc-group/mhc-fine.git")

Environment setup

In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c bioconda kalign3
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install Bio
!pip install --upgrade --no-cache-dir gdown

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - kalign3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    kalign3-3.4.0              |       hdbdd923_0         125 KB  bioconda
    ope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 8.2 MB/s eta 0:00:00


After all libraries are installed, the notebook will be restarted. Just continue running the following cells.


Libraries

In [1]:
import torch
if not torch.cuda.is_available():
    print("Please check your setup of GPU.")

Please check your setup of GPU.


In [2]:
cd /content/mhc-fine

/content/mhc-fine


In [3]:
from src import preprocess, model
import pandas as pd
import gdown
import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
#load the model
model_path = "data/model/mhc_fine_weights.pt"
if not os.path.exists(model_path):
    file_id = "1gZkMGOhwXAHAmTCpR5Azd7lzkW0s-nlK"
    gdown.download(f"https://drive.google.com/uc?id={file_id}", model_path)

Downloading...
From (original): https://drive.google.com/uc?id=1gZkMGOhwXAHAmTCpR5Azd7lzkW0s-nlK
From (redirected): https://drive.google.com/uc?id=1gZkMGOhwXAHAmTCpR5Azd7lzkW0s-nlK&confirm=t&uuid=41f304b2-ecc5-443c-9c43-19f4e577b913
To: /content/mhc-fine/data/model/mhc_fine_weights.pt
100%|██████████| 388M/388M [00:03<00:00, 98.2MB/s]


## Input your data


In [5]:
unique_id1 = "RA-Wild"
protein_sequence1 = "GSHSMRYFHTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREEPRAPWIEQEGPEYWDRETQICKAKAQTDREDLRTLLRYYNQSEAGSHTLQNMYGCDVGPDGRLLRGYHQDAYDGKDYIALNEDLSSWTAADTAAQITQRKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGL"
peptide_sequence = "IWMGKMQKK"

In [6]:
unique_id2 = "RA-Mutant"
protein_sequence2 = "GSHSMRYFDTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREEPRAPWIEQEGPEYWDRETQICKAKAQTDREDLRTLLRYYNQSEAGSHTLQNMYGCDVGPDGRLLRGYHQDAYDGKDYIALNEDLSSWTAADTAAQITQRKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGL"
peptide_sequence = "IWMGKMQKK"

## Get the MSA data

Run part to query the MSA data from the database.

In [7]:
!chmod +x a3m_generation/msa_run

In [8]:
a3m_path1 = f"/content/mhc-fine/a3m_generation/{unique_id1}.a3m"
preprocess.get_a3m(protein_sequence1, a3m_path1, unique_id1)


In [9]:
a3m_path2 = f"/content/mhc-fine/a3m_generation/{unique_id2}.a3m"
preprocess.get_a3m(protein_sequence2, a3m_path2, unique_id2)

## Preprocess the data

In [10]:
np_sample1 = preprocess.preprocess_for_inference(protein_sequence1, peptide_sequence, a3m_path1)

Reading a3m file...
Processing protein chain...
GSHSMRYFHTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREEPRAPWIEQEGPEYWDRETQICKAKAQTDREDLRTLLRYYNQSEAGSHTLQNMYGCDVGPDGRLLRGYHQDAYDGKDYIALNEDLSSWTAADTAAQITQRKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGL
Processing peptide chain...
IWMGKMQKK
Mering features...


In [11]:
np_sample2 = preprocess.preprocess_for_inference(protein_sequence2, peptide_sequence, a3m_path2)

Reading a3m file...
Processing protein chain...
GSHSMRYFDTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREEPRAPWIEQEGPEYWDRETQICKAKAQTDREDLRTLLRYYNQSEAGSHTLQNMYGCDVGPDGRLLRGYHQDAYDGKDYIALNEDLSSWTAADTAAQITQRKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGL
Processing peptide chain...
IWMGKMQKK
Mering features...


## Run AlphaFold, display metrics and save prediction

In [14]:
my_model = model.Model()

FileNotFoundError: [Errno 2] No such file or directory: 'model_path'

In [14]:
my_model.inference(np_sample1, unique_id1)
my_model.inference(np_sample2, unique_id2)

Running inference...
Writing predicted structure:  ./output/RA-Wild.pdb


{'mean_plddt': 96.37762451171875, 'mean_masked_plddt': 79.95874701605902}

In [15]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *
from pyrosetta.teaching import *
init()

PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1018966150 seed_offset=0 real_seed=-1018966150
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1018966150 RG_type=mt19937
PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f06

In [16]:
protein1 = pyrosetta.pose_from_pdb("output/RA-Wild.pdb")
protein2 = pyrosetta.pose_from_pdb("output/RA-Mutant.pdb")
scorefxn = get_fa_scorefxn()
print(scorefxn(protein1))
print(scorefxn(protein2))

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.928125 seconds.
core.import_pose.import_pose: File 'output/RA-Wild.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 266
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LYS:CtermProteinFull 275
core.conformation.Conformation: Found disulfide between residues 101 164
core.conformation.Conformation: Found disulfide between residues 203 259
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis